In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xinit

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [ ]:
import numpy as np
from collections import deque
import random

In [ ]:
import gym

## Build Q(s,a)

- input : state
- output : Q-values for each action

In [ ]:
state_size = 4
action_size = 2
hdim = 15

In [ ]:
class DQNAgent():
    
    def __init__(self, state_size=4, action_size=2, hdim=15, epsilon=1.):
        
        self.epsilon = epsilon
        self.action_size = action_size
        self.experience = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        
        tf.reset_default_graph()
        self.state_ = tf.placeholder(tf.float32, shape=[state_size], name='states')
        self.target_ = tf.placeholder(tf.float32, shape=[action_size], name='targets')
        W1 = tf.get_variable('W1', dtype=tf.float32,
                             shape=[state_size, hdim], initializer=xinit() )
        b1 = tf.get_variable('b1', dtype=tf.float32,
                             shape=[hdim], initializer=xinit() )
        h = tf.nn.relu(tf.matmul(
            tf.expand_dims(self.state_, axis=0), W1) + b1)
        W2 = tf.get_variable('W2', dtype=tf.float32,
                             shape=[hdim, action_size], initializer=xinit() )
        b2 = tf.get_variable('b2', dtype=tf.float32,
                             shape=[action_size], initializer=xinit() )
        self.q_out = tf.reshape(tf.matmul(h, W2) + b2, [action_size])
        self.loss = tf.reduce_mean(tf.pow(self.q_out - self.target_, 2))
        self.update = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.loss)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        

    def get_action(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(0, self.action_size)
        
        q_values = self.sess.run(self.q_out, feed_dict = {
            self.state_ : s.reshape([state_size])
        })
        
        return np.argmax(q_values)
    
    def remember(self, s0, a0, r0, s1, done):
        self.experience.append((s0,a0,r0,s1,done))
        
    def exp_replay(self, batch_size):
        batch = random.sample(self.experience, batch_size)
        for s,a,r,s1,done in batch:
            s = s.reshape([state_size])
            target = r
            if not done:
                q1 = self.sess.run(self.q_out, feed_dict = {
                    self.state_ : s1
                })
                q1_max = np.max(q1)
                target = r + (self.gamma * q1_max)
            q = self.sess.run(self.q_out, feed_dict = {
                self.state_ : s
            })
            q[a] = target
            self.sess.run(self.update, feed_dict = {
                self.state_ : s,
                self.target_ : q
            })
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
        # sort memories
        self.experience = sorted(self.experience, key=lambda x : x[2])

In [ ]:
env = gym.make('CartPole-v0')

[2017-06-05 16:15:39,961] Making new env: CartPole-v0


In [ ]:
agent = DQNAgent()

In [ ]:
episodes = 5000
for e in range(episodes):
    
    s = env.reset()
    
    for t in range(500):
        a = agent.get_action(s)
        s1, r, done, _ = env.step(a)
        # remember
        agent.remember(s,a,r,s1,done)
        # update state
        s = s1
        
        if done:
            print("episode: {}/{}, score: {}".format(e, episodes, t))
            break
    
    batch_size = 128
    if len(agent.experience) > batch_size :
        agent.exp_replay(batch_size)        

episode: 0/5000, score: 10
episode: 1/5000, score: 22
episode: 2/5000, score: 25
episode: 3/5000, score: 15
episode: 4/5000, score: 37
episode: 5/5000, score: 11
episode: 6/5000, score: 17
episode: 7/5000, score: 13
episode: 8/5000, score: 16
episode: 9/5000, score: 17
episode: 10/5000, score: 12
episode: 11/5000, score: 18
episode: 12/5000, score: 17
episode: 13/5000, score: 13
episode: 14/5000, score: 15
episode: 15/5000, score: 9
episode: 16/5000, score: 27
episode: 17/5000, score: 16
episode: 18/5000, score: 10
episode: 19/5000, score: 18
episode: 20/5000, score: 15
episode: 21/5000, score: 13
episode: 22/5000, score: 13
episode: 23/5000, score: 16
episode: 24/5000, score: 16
episode: 25/5000, score: 11
episode: 26/5000, score: 25
episode: 27/5000, score: 10
episode: 28/5000, score: 31
episode: 29/5000, score: 18
episode: 30/5000, score: 17
episode: 31/5000, score: 13
episode: 32/5000, score: 11
episode: 33/5000, score: 37
episode: 34/5000, score: 15
episode: 35/5000, score: 9
epis

episode: 287/5000, score: 61
episode: 288/5000, score: 131
episode: 289/5000, score: 92
episode: 290/5000, score: 96
episode: 291/5000, score: 199
episode: 292/5000, score: 194
episode: 293/5000, score: 199
episode: 294/5000, score: 56
episode: 295/5000, score: 82
episode: 296/5000, score: 199
episode: 297/5000, score: 117
episode: 298/5000, score: 88
episode: 299/5000, score: 199
episode: 300/5000, score: 199
episode: 301/5000, score: 122
episode: 302/5000, score: 180
episode: 303/5000, score: 111
episode: 304/5000, score: 199
episode: 305/5000, score: 187
episode: 306/5000, score: 147
episode: 307/5000, score: 145
episode: 308/5000, score: 199
episode: 309/5000, score: 170
episode: 310/5000, score: 177
episode: 311/5000, score: 199
episode: 312/5000, score: 98
episode: 313/5000, score: 181
episode: 314/5000, score: 199
episode: 315/5000, score: 114
episode: 316/5000, score: 146
episode: 317/5000, score: 199
episode: 318/5000, score: 157
episode: 319/5000, score: 199
episode: 320/5000

episode: 561/5000, score: 199
episode: 562/5000, score: 199
episode: 563/5000, score: 199
episode: 564/5000, score: 199
episode: 565/5000, score: 199
episode: 566/5000, score: 199
episode: 567/5000, score: 199
episode: 568/5000, score: 199
episode: 569/5000, score: 199
episode: 570/5000, score: 188
episode: 571/5000, score: 199
episode: 572/5000, score: 199
episode: 573/5000, score: 199
episode: 574/5000, score: 199
episode: 575/5000, score: 199
episode: 576/5000, score: 199
episode: 577/5000, score: 199
episode: 578/5000, score: 199
episode: 579/5000, score: 199


KeyboardInterrupt: 

In [ ]:
avg_reward = 0

for i in range(100):
    
    s = env.reset()
    
    for t in range(500):
        a = agent.get_action(s)
        s1, r, done, _ = env.step(a)
        # remember
        #agent.remember(s,a,r,s1,done)
        # update state
        s = s1
        if done:
            avg_reward += t
            break
print(avg_reward/100)

197.43
